# Building ML Classifiers

In [1]:
# Ideas: try simpler models first, such as logistic regression, SVM classifier, ...
# Here I want to focu on ensemble models to learn more about them along the way

### 1) Random Forest model

In [2]:
# Read in & clean text
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("DataCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Explore RandomForestClassifier Attributes & Hyperparameters

In [3]:
from sklearn.ensemble import RandomForestClassifier

print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_estimator_type', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_required_parameters', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0

### Cross-Validation

In [4]:
from sklearn.model_selection import KFold, cross_val_score

In [5]:
rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf, X_count_feat, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

array([0.96678636, 0.97307002, 0.97304582, 0.95687332, 0.96495957])

### test on the holdout set

In [6]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_count_feat, data['label'], test_size=0.2)

In [8]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)

In [9]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:10]

[(0.0818705362419516, 'body_len'),
 (0.04456223813178019, 3134),
 (0.03481455042388913, 7350),
 (0.027070415625805193, 4796),
 (0.025816900189876705, 5724),
 (0.02536407267751236, 7218),
 (0.022522429298180616, 1803),
 (0.020991871915145198, 1881),
 (0.018424528482627153, 1361),
 (0.017257876644248868, 2031)]

In [10]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

In [11]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

Precision: 1.0 / Recall: 0.569 / Accuracy: 0.944


### Explore Random Forest model with grid-search

In [12]:
# Here I make my own grid search instead of using sklearn toolboxes for it

In [13]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision, 3), round(recall, 3),
        round((y_pred==y_test).sum() / len(y_pred), 3)))

In [14]:
# In practice, I would need to search in a bigger range, 
# here I will focus on a small range:
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.215 / Accuracy: 0.899
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.667 / Accuracy: 0.957
Est: 10 / Depth: 30 ---- Precision: 1.0 / Recall: 0.667 / Accuracy: 0.957
Est: 10 / Depth: None ---- Precision: 0.974 / Recall: 0.771 / Accuracy: 0.968
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.257 / Accuracy: 0.904
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.59 / Accuracy: 0.947
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.701 / Accuracy: 0.961
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.826 / Accuracy: 0.978
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.222 / Accuracy: 0.899
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.556 / Accuracy: 0.943
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.667 / Accuracy: 0.957
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.84 / Accuracy: 0.979


In [ ]:
# Lets try GridSearchCV on both features of tf_idf and countvectorizer
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

In [ ]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]